### Formateado de la variable Total Charges a numércia

In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/Users/gestion/OneDrive/Desktop/churn/data/raw.csv')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Revisar si se convirtió
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [7]:
# Volvemos a revisar los nulos por si al convertir perdimos datos
def detectar_nulos_personalizados(df, valores_nulos=None):
    if valores_nulos is None:
        valores_nulos = ['NA', 'Na', 'na', 'NN', 'nn', 'Nn', 'None', 
                        'NONE', 'none', 'N', 'n', '', ' ', '-']
    
    resultados = []
    
    for col in df.columns:
        nulos_standard = df[col].isnull().sum()
        
        nulos_personalizados = 0
        if df[col].dtype == 'object':
            nulos_personalizados = df[col].isin(valores_nulos).sum()
        
        total_nulos = nulos_standard + nulos_personalizados
        porcentaje = (total_nulos / len(df)) * 100
        
        resultados.append({
            'Columna': col,
            'Total_Nulos': total_nulos,
            'Porcentaje': round(porcentaje, 2)
        })
    
    df_nulos = pd.DataFrame(resultados)
    df_nulos = df_nulos.sort_values('Total_Nulos', ascending=False)
    df_nulos = df_nulos.reset_index(drop=True)
    
    return df_nulos

In [ ]:
# Comprobamos los nulos y le imputamos la mediana
detectar_nulos_personalizados(df)
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

C:\Users\gestion\AppData\Local\Temp\ipykernel_17432\4052900489.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


In [13]:
# Comprobamos que ya no hay nulos
detectar_nulos_personalizados(df)

,Columna,Total_Nulos,Porcentaje
0,customerID,0,0.0
1,gender,0,0.0
2,SeniorCitizen,0,0.0
3,Partner,0,0.0
4,Dependents,0,0.0
5,tenure,0,0.0
6,PhoneService,0,0.0
7,MultipleLines,0,0.0
8,InternetService,0,0.0
9,OnlineSecurity,0,0.0


### Tratamiento de variables categóricas correlacionadas

Las variables a tratar son las siguientes: **InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV y StreamingMovies**
Como vemos, las variables OnlineSecurity, OnlineBackup, DeviceProtection y TechSupport pueden ofrecer información muy similar, ya que todas tienen componentes de seguridad y confianza acerca de los servicios ofrecidos por la compañía. Por lo tanto, nos quedaremos tan sólo con una de ellas.

En cuanto a InternetService, la matendremos debido a que no se parece mucho a ninguna otra variable con la que está correlacionada.

Para las variables de Streaming, tan sólo nos quedaremos con una, ya que los servicios muchas veces son similares.

In [15]:
# Dejamos los campos más genéricos
cols_to_drop = [
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingMovies'
]

df = df.drop(columns=cols_to_drop)

df.columns


Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'StreamingTV', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [20]:
df.to_csv('df_clean.csv', index=False)
